# PRIMEROS PASOS EN QISKIT

En este notebbok vamos a aprender a usar Qiskit para definir un circuito sencillo y ejecutarlo tanto en simuladores como en los ordenadores cuánticos de la IBM Quantum Experience. 

Comenzamos importando los paquetes necesarios.

In [ ]:
%matplotlib inline

from qiskit import *
from qiskit.visualization import *
from qiskit.tools.monitor import *

## Definiendo el circuito

Ahora, vamos a definir un circuito muy sencillo: usaremos una puerta $H$ para poner un qubit en un estado de superposición y, a continuación, lo mediremos

In [ ]:
#Vamos a hacer un circuito para crear un estado de superposición y medirlo

circ = QuantumCircuit(1,1) # Usaremos un qubit y un bit clásico para medir el resultado

circ.h(0) #Aplicamos la puerta H al qubit

circ.measure(range(1),range(1)) # Medimos

circ.draw(output='mpl') #Dibujamos el circuito

También podemos obtener, fácilmente, el código *qasm* correspondiente.

In [ ]:
print(circ.qasm())

## Ejecutando en distintos simuladores locales

Una vez definido el circuito, podemos ejecutarlo en un simulador de forma local. 

In [ ]:
# Ejecución en el simulador local

backend = Aer.get_backend('aer_simulator') # Elegimos el backend para ejecutar

job = execute(circ, backend, shots=1024) # Ejecutamos, seleccionando el número de repeticiones

result = job.result() # Recogemos los resultados

counts = result.get_counts(circ) # Obtenemos las frecuencias de cada resultado y las mostramos
print(counts) 
plot_histogram(counts)

También podemos realizar una simulación en la que obtenemos el vector de estado final. Para ello, definimos un nuevo circuito con una puerta $H$ pero sin la medida y con una instrucción para guardar el vector de estado

In [ ]:
#Ejecución para obtener el vector de estado

circ2 = QuantumCircuit(1,1)

circ2.h(0)
circ2.save_statevector()

job = execute(circ2, backend) # Ejecutamos

result = job.result() #Recogemos los resultados y recuperamos el vector de estado
outputstate = result.get_statevector(decimals=3)
print(outputstate)

Finalmente, también podemos obtener la matriz unitaria que representa la evolución del estado en el circuito. 

In [ ]:
circ3 = QuantumCircuit(1,1)

circ3.h(0)
circ3.save_unitary()

job = execute(circ3, backend) # Ejecutamos

result = job.result() #Recogemos los resultados y recuperamos la matriz unitaria
unitary = result.get_unitary(decimals = 3)
print(unitary)

Ahora vamos a conectar con los ordenadores de la IBM Quantum Experience para poder lanzar nuestros trabajos en ellos.

In [ ]:
# Conectando con los ordenadores cuánticos reales

provider = IBMQ.load_account() # Cargamos la cuenta 
provider.backends() # Recuperamos los backends disponibles y comprobamos su estado

for b in provider.backends():
    print(b.status().backend_name, "tiene", b.status().pending_jobs, 
          "trabajos pendientes. Status:",b.status().status_msg)

Podemos ejecutar en el simulador que nos ofrece la IBM Quantum Experience (hasta 32 qubits). Sólo hace falta cambiar el backend y ejecutar como habíamos hecho antes.

In [ ]:
# Ejecución en el simulador de la IBM Q Experience

backend_sim = provider.get_backend('ibmq_qasm_simulator') # Elegimos el backend para ejecutar

job_sim = execute(circ, backend_sim, shots=1024) # Ejecutamos, seleccionando el número de repeticiones

result_sim = job_sim.result() # Recogemos los resultados

counts = result_sim.get_counts(circ) # Obtenemos las frecuencias de cada resultado y las mostramos
print(counts) 
plot_histogram(counts)

Para ejecutar en un ordenador cuántico real, sólo es necesario cambiar el backend. Usaremos *job_monitor* para observar el progreso del trabajo lanzado.

In [ ]:
# Ejecutando en el ordenador cuántico

backend = provider.get_backend('ibmq_armonk')

job_exp = execute(circ, backend=backend)
job_monitor(job_exp)

Una vez finalizado el trabajo, podemos recoger los resultados y mostrarlos, comparándolos con los obtenidos en el simulador.

In [ ]:
result_exp = job_exp.result()
counts_exp = result_exp.get_counts(circ)
plot_histogram([counts_exp,counts], legend=['Device', 'Simulator'])